In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression, f_classif, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from scipy.stats import pearsonr

path = Path('')



In [38]:
# Read the data into a Pandas dataframe
df = pd.read_parquet(path='./outage_data.parquet', engine='pyarrow')

# Remove duplicate entries in 2019
# Remove all rows with SimStartDate after 2019-01-01 and event_type == 'thunderstorm'
df = df.loc[~((df['SimStartDate'] > '2019-01-01') & (df['event_type'] == 'thunderstorm'))]

# Correlation Threshold

In [39]:
df.drop(columns = ['point_ewkt'], inplace = True)

# apply ordinal encoding to 'poly_ewkt', 'point_ewkt', 'event_type' columns
non_numerical_columns = ['poly_ewkt', 'event_type']

for column in non_numerical_columns:
    df[column] = df[column].astype('category')
    df[column] = df[column].cat.codes

# Convert datetime columns to separate columns for year, month, day, hour, minute, second
datetime_features = list(df.select_dtypes(include = "datetime64[ns, UTC]").columns)
for i in datetime_features:
    df[i+"_year"] = df[i].dt.year
    df[i+"_month"] = df[i].dt.month
    df[i+"_day"] = df[i].dt.day
    df[i+"_hour"] = df[i].dt.hour

# Use all data after Nov 1, 2018 (15 storms) as test set
test_df = df.loc[df['SimStartDate'] >= '2018-11-01']
train_df = df.loc[df['SimStartDate'] < '2018-11-01']

# Drop columns that are not needed for training
train_df = train_df.drop(datetime_features, axis=1)
test_df = test_df.drop(datetime_features, axis=1)

# Droplist - 90%

In [47]:
import pickle

droplist = pickle.load(open('drop_list.pkl', 'rb'))
train_df.drop(columns = droplist, inplace = True)

In [ ]:
# target = df['outage_count']
# cm_df = df.drop(['outage_count'], axis=1)
threshold = 0.3
feature_list = []
cm_df = train_df.copy()

target = cm_df['outage_count']
cm_df = cm_df.drop(['outage_count'], axis=1)

count = 0
for col in cm_df.columns:
    # print("col: ", col, "dtype: ", cm_df[col].dtype)
    if cm_df[col].dtype != 'object' and cm_df[col].dtype != 'datetime64[ns]' and cm_df[col].dtype != 'datetime64[ns, UTC]':
        count += 1
        # print(cm_df[col].dtype)
        label = cm_df[col]
        corr = label.corr(target)
        # print(corr)
        if abs(corr) > threshold:
            feature_list.append(col)
            print("col: ", col, "correlation: ", corr)

# Lasso

In [ ]:
# Import necessary libraries
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assume you have your data in X (features) and y (target)
# Split the data into training and testing sets
X = train_df.drop(['outage_count'], axis=1)
y = train_df['outage_count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for regularization methods)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a Lasso regression model
lasso = Lasso(alpha=0.01)  # You can adjust the alpha parameter to control the strength of regularization

# Fit the model on the training data
lasso.fit(X_train_scaled, y_train)

# Get the selected features
selected_features = np.where(lasso.coef_ != 0)[0]

# Print or use the selected features
print("Selected Features:", selected_features)

# Evaluate the model on the test set
accuracy = lasso.score(X_test_scaled, y_test)
print("Model Accuracy:", accuracy)


In [ ]:
len(selected_features)

In [ ]:
# feature_list_index = selected_features.tolist()
# X_train = X_train.iloc[:, feature_list_index]

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Standardize the features (important for regularization methods)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Create a Lasso regression model
# lasso = Lasso(alpha=0.05)  # You can adjust the alpha parameter to control the strength of regularization

# # Fit the model on the training data
# lasso.fit(X_train_scaled, y_train)

# # Get the selected features
# # selected_features = np.where(lasso.coef_ != 0)[0]

# # Print or use the selected features
# # print("Selected Features:", selected_features)

# # Evaluate the model on the test set
# accuracy = lasso.score(X_test_scaled, y_test)
# print("Model Accuracy:", accuracy)

# Chi2

In [48]:
# retrieve categorical features
def check_categorical(df, max_unique_values=10):
    """
    This function checks each column in the given dataframe to ascertain if it is categorical.
    It is based on the number of unique values and the data type of the column.

    :param df: Pandas DataFrame containing the data to check
    :param max_unique_values: The maximum number of unique values to consider a feature as categorical
    :return: A dictionary with the column name as keys and a boolean indicating if it is categorical as values
    """
    categorical = {}
    for column in df.columns:
        # Check if the data type is object or if there are few unique values relative to the size of the dataset
        if df[column].dtype == 'object' or len(df[column].unique()) <= max_unique_values:
            categorical[column] = True
        else:
            # Check for numeric columns that are actually categorical (like IDs or codes)
            # Here, we'll consider an integer column with a low number of unique values as categorical
            if df[column].dtype in ['int64', 'int32'] and df[column].nunique() < max_unique_values:
                categorical[column] = True
            else:
                categorical[column] = False
    return categorical

categorical = check_categorical(train_df, max_unique_values=10)
categorical = {k: v for k, v in categorical.items() if v}
categorical_list = list(categorical.keys())
categorical_list

['event_type',
 '220_220 (max)_lambert_levels 0-40000-filtered_24-max_min',
 'SimStartDate_year',
 'SimStartDate_hour',
 'outage_start_time_year',
 'outage_start_time_hour',
 'outage_end_time_year',
 'outage_end_time_hour',
 'weather_start_time_year',
 'weather_start_time_hour',
 'weather_end_time_year',
 'weather_end_time_hour']

In [49]:
from sklearn.feature_selection import SelectKBest

X = train_df.drop(['outage_count'], axis=1)
y = train_df['outage_count']

X_categorical = X[categorical_list]
### S0LUTION FUNCTION
# ADD YOUR SOLUTION TO THE DESIGNATED AREA
# DO NOT ALTER THE SURROUNDING CODE

def cat_feature_selection(cat_data,y):
    ### BEGIN SOLUTION ###
    ftest = SelectKBest(score_func=chi2, k='all')
    ftest.fit(cat_data, y)
    fscores = pd.DataFrame(ftest.scores_)
    dfcolumns = pd.DataFrame(cat_data.columns)
    featureScores = pd.concat([dfcolumns, fscores], axis=1)
    featureScores.columns = ['Feature', 'Score']
    featureScores = featureScores.sort_values('Score', ascending=False)
    return featureScores

featureScores = cat_feature_selection(X_categorical,y)

In [50]:
featureScores

,Feature,Score
3,SimStartDate_hour,306.204227
5,outage_start_time_hour,306.204227
7,outage_end_time_hour,306.204227
9,weather_start_time_hour,306.204227
11,weather_end_time_hour,306.204227
0,event_type,45.659868
1,220_220 (max)_lambert_levels 0-40000-filtered_...,12.939021
2,SimStartDate_year,0.032677
4,outage_start_time_year,0.032677
6,outage_end_time_year,0.032677


In [51]:
useless_cat_faetures = featureScores[featureScores['Score'].isna()]['Feature'].tolist()
useless_cat_faetures

[]

In [45]:
len(useless_cat_faetures)

422

In [46]:
import pickle
with open('drop_list_anova_cat.pkl', 'wb') as f:
    pickle.dump(useless_cat_faetures, f)

In [53]:
len(pickle.load(open('drop_list_anova_cat.pkl', 'rb')))


422